In [ ]:
#| include: false

import os
import pandas as pd
import numpy as np
import sqlite3
# import plotly.express as px
import toml
from pathlib import Path
import summary_data

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS
input_config = summary_data.INPUT_CONFIG

## Parcel Updates
Adding Military Jobs to Parcels

# create_ixxi_work_trips.py
### Vehicle trips to/from JBLM zones by origin/destination 

In [157]:
df = pd.DataFrame(results_df[jblm_taz_list].sum(), columns=['trips_to_jblm'])
df_from = pd.DataFrame(results_df.loc[jblm_taz_list].sum(axis=1), columns=['trips_from_jblm'])
df = df.merge(df_from, left_index=True, right_index=True, how='outer')

# Display with commas and as integer
df = df.sort_values(by='trips_from_jblm', ascending=False)
df['trips_total'] = df['trips_to_jblm'] + df['trips_from_jblm']
df.loc['Total'] = df.sum(numeric_only=True)
for col in df.columns:
    df[col] = df[col].apply(lambda x: f"{int(x):,}")

df


,trips_to_jblm,trips_from_jblm,trips_total
3351,"16,557","16,557","33,114"
3070,"10,975","10,975","21,950"
3353,"7,302","7,302","14,604"
3355,"7,162","7,162","14,325"
3349,"6,091","6,091","12,182"
3350,"2,944","2,944","5,889"
3354,"2,735","2,735","5,470"
3352,"1,561","1,561","3,123"
3346,"1,180","1,180","2,361"
3356,984,984,"1,969"


In [ ]:
# Total trips from external stations
results_df[3700:].sum().sum()

np.float64(83916.24721269442)

In [166]:
# Total trips to external stations
results_df.loc[3700:].sum().sum()

np.float64(83927.1946450092)

#### Daysim IXXI File

This file in generated in create_ixxi_work_trips.py and specifies what share of each zone to exclude from DaySim Work Location model

Within JBLM area, Jobs_XIFrac should be 1 because we are using the exogenous data for work flows in these zones
If Jobs_XIFrac == 1 then Daysim will not send workers to jobs in these zones

Worker_IXFrac is not equal to 1 because workers in the synthetic population for these areas 
may have jobs outside the base area (family members of enlisted)


In [84]:
# Load ixxi fractions file and set column names

df_ixxi = pd.read_csv(r"../../../../outputs/landuse/psrc_worker_ixxifractions.dat", sep='\t', header=None)
df_ixxi.columns = ['taz', 'Worker_IXFrac', 'Jobs_XIFrac']
df_ixxi[df_ixxi['taz'].isin(jblm_taz_list)]

,taz,Worker_IXFrac,Jobs_XIFrac
3060,3061,0.011,1.0
3069,3070,1.000,1.0
3345,3346,0.000,1.0
3347,3348,0.010,1.0
3348,3349,0.006,1.0
3349,3350,0.083,1.0
3350,3351,1.000,1.0
3351,3352,0.024,1.0
3352,3353,0.965,1.0
3353,3354,0.000,1.0


### Non-Work

Trip Generation

In [232]:
# Productions from external zones
df = pd.read_csv('../../../../outputs/supplemental/7_balance_trip_ends.csv')
prods = int(df.loc[df['taz'] >= 3733, 'hsppro'].sum())
print(f"Total productions from external zones: {prods:,}")


Total productions from external zones: 75,111


In [214]:
# Trip Generation
# for non-work trips
df = pd.read_csv('../../../../outputs/supplemental/7_balance_trip_ends.csv')
df = df[df['taz'].isin(jblm_taz_list)][['hsppro']].sort_values(by='hsppro', ascending=False)
df.loc['Total'] = df.sum(numeric_only=True)
# show with commas and as integer
df['hsppro'] = df['hsppro'].apply(lambda x: f"{int(x):,}")
# Create total row

df

,hsppro
3069,"1,392"
3350,"1,110"
3349,"1,031"
3347,974
3351,832
3348,643
3060,426
3354,28
3352,18
3355,10


Trip Tables